In [1]:
import time,cv2,os,glob
from datetime import datetime
import numpy as np
import IPython.display as Disp
os.sys.path.append("../utilities/")
import support_utility_openvino
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import xml

In [2]:
person_detection_model_path = '/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/faster_rcnn_inception_v2_coco/faster_rcnn_inception_v2_coco_2018_01_28/openvino_graph/frozen_inference_graph.xml'
num_requests = 4
# base_xml_path = config.BASE_ANNOTATION_XML_PATH
channel = '/media/acer/shared/aws-s3/dna-computer-vision/COMPUTER_VISION/SORTED_DATASET/HINDALCO_TRIAL/CLADDING/TALOJA/data/INGOT_TILTER_VIEW/18_02_2021/test_videos/Wrong Arrow detection Trail 3 CROPPED.mp4'

In [3]:
person_detection_model = support_utility_openvino.async_infer(4)
person_detection_model.load_model(model_path=person_detection_model_path,device="CPU")
output_support = support_utility_openvino.create_plot()

Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_info', 'image_tensor'])
model outputs :  dict_keys(['detection_output'])


IndexError: list index out of range

In [14]:
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [15]:
tree = ET.parse(base_xml_path)
xmlRoot = tree.getroot()

In [17]:
person_detection_model.reinit_model()
# helmet_vest_model.reinit_model()
fps = []
thresh_person = .2
thresh_hv = .2
cap = cv2.VideoCapture(channel)
offset = 20
resolution_thresh_range =[0.001,.4] ##percentage threshold(on resolution) for person detection

write_video = False
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("output.mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))

COUNT_ID = 2

image = None

while True:
    ret,frame = cap.read()
    #print("Frame no: ", cap.get(1))
    frame_no = int(cap.get(1))
    if not ret:
        break
    masked_frame = frame.copy()
    
    frame_name = 'frame_'+str(frame_no).zfill(6)
    
    t1 = time.time()
    person_detection_model.predict(masked_frame,attr=frame)
    if person_detection_model.frame_processed>=person_detection_model.num_requests:
        masked_frame,orig_frame,res_person = person_detection_model.postprocess_op()
        cropped_frames,updated_bbox,res_person_filt = output_support.trim_frame_with_result(orig_frame,res_person[0],offset=offset,\
                                                                                            return_results=True,threshold=thresh_person,resolution_thresh_range =resolution_thresh_range )
        
        h, w = orig_frame.shape[:2]
        
        image = ET.Element('image')
        image.attrib['id'] = str(frame_no)
        image.attrib['name'] = frame_name
        image.attrib['width'] = str(w)
        image.attrib['height'] = str(h)

        cv2.putText(orig_frame, "Frame Count: "+str(frame_no), (w-1000, 100), 1, 2, (255, 0, 233), 3)
        
        for b in updated_bbox:
            x1 = b[0]
            y1 = b[1]
            x2 = b[2]
            y2 = b[3]
            
            
            box = ET.SubElement(image, 'box')
            box.attrib['label'] = "person"
            box.attrib['occluded'] = "0"
            box.attrib['xtl'] = str(float(x1))
            box.attrib['ytl'] = str(float(y1))
            box.attrib['xbr'] = str(float(x2))
            box.attrib['ybr'] = str(float(y2))
            
            cv2.rectangle(orig_frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
            
        xmlRoot.append(image)
        cv2.imshow('person_frame', orig_frame)
        k = cv2.waitKey(1)
        if k == 27:
            cv2.destroyAllWindows()
            break
            
    COUNT_ID += 1
cv2.destroyAllWindows()

In [9]:
indent(xmlRoot)

In [10]:
tree.write(config.PERSON_ANNOTATION_XML_PATH)

In [1]:
from openvino.inference_engine import IENetwork,IECore

In [2]:
path_to_xml_file = '/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/faster_rcnn_inception_v2_coco/faster_rcnn_inception_v2_coco_2018_01_28/openvino_graph/frozen_inference_graph.xml'
path_to_bin_file = '/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/faster_rcnn_inception_v2_coco/faster_rcnn_inception_v2_coco_2018_01_28/openvino_graph/frozen_inference_graph.bin'

In [3]:
ie = IECore()
net = ie.read_network(model=path_to_xml_file, weights=path_to_bin_file)
exec_net = ie.load_network(network=net, device_name="CPU", num_requests=2)

In [4]:
input_blob = (net.input_info))
out_blob = list(net.outputs.keys())

In [13]:
list(net.inputs.keys())

/home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  """Entry point for launching an IPython kernel.


['image_info', 'image_tensor']

In [8]:
net.input_info['image_tensor']

In [5]:
blob_shape = net.input_info[input_blob].tensor_desc.dims

In [6]:
blob_shape

[1, 3]

In [10]:
net.input_info['image_tensor'].tensor_desc.layout

'NCHW'

In [21]:
list()

'image_info'

In [22]:
out_blob

['detection_output']